<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Tensorflow/Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# コールバークの使い方
* コールバックは訓練中で適用される関数集合です。
    * 訓練中に**モデル内部の状態と統計量を可視化する**際に、コールバックを使います。
* SequetialとModelクラスの.fit()メソッドに(キーワード引数callbacksとして)コールバックのリストを渡すことができます。

## Callback
* keras.callbacks.Callback()
* 新しいコールバックを構築するために使用されます
    * params: 辞書、訓練のパラメタ(冗長性、バッチサイズ、epoch数)
    * model: keras.models.Modelのインスタンス。
    * コールバック関数が引数として取る辞書のlogsは、現在のバッチ数かエポック数に関連したデータのキーを含みます。
    * 現在、Sequentialモデルクラスの.fit()メソッドは、そのコールバックに渡すlogsに以下のデータが含まれます
    * on_epoch_end：ログはaccとlossと含み、val_loss, val_accを含みます。
    * on_batch_begin：ログは現在のバッチのサンプル数sizeを含みます。
    * on_batch_end：ログはlossとオプションとしてaccを含みます。


In [ ]:
callback = keras.callbacks.Callback()

## BaseLogger
* 監視されている評価値のエポック平均を蓄積するコールバックです
* このコールバックは全kerasモデルに自動的に適用されます

In [ ]:
baselogger = keras.callbacks.BaseLogger()

## TerminateOnNaN
* 損失がNaNになった時に訓練を終了するコールバックです

In [ ]:
terminateonNaN = keras.callbacks.TerminateOnNa()

## ProgbarLogger
* 標準出力に**評価値を出力するコールバック**
* 引数
    * count_mode: "steps"か"samples"の一方、
    * サンプルかステップ(バッチ)のどちらかをプログレスバーの集計に使うか。
* Raises
    * ValueError: count_modeの値が不正の時

In [ ]:
progbarlogger = keras.callbacks.ProgbarLogger(count_mode = 'samples')

## History
* Historyオブジェクトにイベントを記録するコールバックです
* このコールバックは全Kerasモデルに自動的に適用されます。
    * Historyオブジェクトはモデルのfitメソッドの戻り値として取得します


In [ ]:
history = keras.callbacks.History()

## ModelCheckpoint
* 各エポック終了後に**モデルを保存**します。
* filepathは、(on_epoch_endで渡された)epochの値とlogsのキーで埋められた書式設定オプションを含むことが出来ます。
* 例えば、filepathが、**weights.{epoch:02d}-{val_loss:.2f}.hdf5の場合、
* 複数のファイルがエポック数とvalidation lossの値を付与して保存されます
    * filepath:文字列、モデルファイルを保存するパス
    * monitor:監視する値
    * verbose:冗長モード 0 or 1
    * save_best_only: **save_best_only=True**の場合、監視しているデータによって、**最新の最良モデルが上書きされません**
    * mode: **{auto, min, max}の内の1つが選択されます。
    * **save_best_only=True**ならば、現在保存されているファイルを上書きするかは、監視されている値の最大化か最小化によって決定されています。
        * val_accの場合、この引数はmax
        * val_lossの場合はmin
        * autoモードでえは、最大化・最小化・いずれかを監視されている値の名前から自動的に推定
    * save_weights_only:Trueなら、モデルの重みがオゾンされます。
        * (**model.save_weights(filepath)).
        * そうでないなら、モデルの全体が保存されます。
            * **model.save(filepath)**
    * period:チェックポイント間の間隔(epochs)

In [ ]:
modelcheckpoint = keras.callbacks.ModelCheckpoint(filepath,
                                                  monitor='val_loss',
                                                  verbose = 0,
                                                  save_best_only=False,
                                                  save_weights_only = False,
                                                  mode = 'auto',
                                                  period = 1)

## EarlyStopping
* 監視する値の変化が停止したときに訓練を終了します。
    * monitor:監視する値。
    * min_delta:監視する値について改善として判定される最小変化値。
        * つまり、min_deltaよりも絶対値の変化が小さければ改善していないとみなします
    * patience:ここで指定したエポック数の間(監視する値に)改善がないと、訓練が停止します
    * verbose:冗長モード
    * mode:{auto, min, max}の内、1つが選択されます。
        * minモードでは、監視する値の**減少が停止したときに**訓練を終了します
        * maxモードでは、監視する値の**増加が停止したときに**訓練おわり
        * autoモードは、自動的に監視されている値から推定！

In [ ]:
earlystopping = keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 0, verbose = 0, mode = 'auto')

## RemoteMonitor
* コールバックはサーバーにイベントをストリームするときに使用
* requestsライブラリが必要です。
* イベントはデフォルトで** root + '/publish/epoch/end/'**に送信されます。
* コールすることによって、イベントデータをJSONエンコードした辞書のdata引数をHTTP POSTされます。
    * root:文字数; 対象サーバーのroot URL
    * path:文字数; イベントを送るrootへの相対パス
    * field:文字列; データを保存するJSONのフィールド
    * headers: 辞書; オプションでカスタムできるHTTPヘッダー

In [ ]:
remotemonitor = keras.callbacks.RemoteMonitor(root='http://なんちゃら', path = '/nanka/', field = 'data', headers = None)

## LearningRateScheduler
* 学習率のScheduler
    * schedule:この関数はエポックのインデックスを入力とし、
        * **新しい学習率**を返します
    * verbose:整数。0:何も表示しない、1:更新メッセージを表示

In [ ]:
lrScheduler = keras.callbacks.LearningRateScheduler(schedule, verbose = 1)

## TensorBoard
* 可視化
    * log_dir:保存するパス
    * histogram_freq:モデルの層の活性化ヒストグラムを計算する**頻度**、
        * この値を0に設定するとヒストグラムが計算されません。
        * ヒストグラムの可視化には**validation dataを指定しておく必要があります**
    * write_graph:TensorBoardのグラフを可視化するか。
        * write_graphがTrueの場合、ログファイルが非常に大きくなることがあります
    * write_grads:TensorBoardに**勾配のヒストグラフを可視化するかどうか**
        * histogram_freqは0より大きくしなければなりません。
    * batch_size:ヒストグラム計算のネットワークに渡す入力のバッチサイズ
    * write_images:TensorBoardで可視化するモデルの重みを画像として下記ガス化どうか？
    * embeddings_freq:選択したembeddingsレイヤーを保存する(epochに対する)頻度
    * embeddings_layer_names:観察するレイヤー名のリスト。
        * もしNoneか空リストなら全embeddingsレイヤーを観察します
    * embeddings_metadata:レイヤー名からembeddingsレイヤーに関するメタデータの保存ファイル名へマップする辞書。
        * すべては文字列を渡します

In [ ]:
tensorflow = keras.callbacks.TensorBoard(log_dir='./logs',
                                         histogram_freq=0,
                                         batch_size=32,
                                         write_graph=True,
                                         write_grads=False,
                                         write_images=False,
                                         embeddings_freq=0,
                                         embeddings_layer_names=None,
                                         embeddings_metadata=None)

* 起動は
 - %load_ext tensorboard
 - %tensorboard --logdir logs ってやる

---

## ReduceLROnPlateau
* 評価値の改善が止まった時に**学習率を減らします**
    * モデルは訓練が停滞したときに学習率を 2 ~ 10 で割ることで恩恵をうけることがあります
    * このコールバックは評価値を監視し、'patience'で指定されたエポック数の間改善が見られなかった場合、学習を減らします。
* options
    * monitor:監視する値
    * factior:学習率を減らす割合, **new_lr = lr * factor**
    * patience:何エポック改善が見られなかったら学習率の削減を行うか
    * verbose:0,何もしない, 1:学習率削減時メッセージを表示
    * mode : auto, min, maxのいずれか
        * min:**減少が停止した際に、**学習率を更新
        * max:**増加が停止したときに、**学習率を更新します
        * auto:名前から判断
    * epslion:改善があったと判断する閾値。**有意義な変化だけに注目するために**用います
    * cooldown: 学習率を減らした後、通常の学習を再開するまで待機するエポック数
    * min_lr : **学習率の下限**

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience = 5,
                              min_lr = 0.001)

In [ ]:
model.fit(X_train, y_train, callbacks = [reduce_lr])

## CSVLogger
* 各エポックの結果を**csvファイルに保存するコールバック**です
* np.ndarrayのような1次元イテラブルを含む、文字列表現可能な値をサポートしています
* option:
    * filename: csvファイル名、
    * separator: csvファイルで各要素を区切るために用いられる文字。
    * append: 
        * True: ファイルが存在する場合、追記します。
        * **(訓練を続ける場合に便利です)**
        * False: 既存のファイルを上書きします
    

In [ ]:
csv_logger = keras.callbacks.CSVLogger('training.log')

In [ ]:
model.fit(X_train, y_train, callbacks = [csv_logger])

## LambdaCallBack
* シンプルな自作コールバックを急いで作るためのコールバックでｓ
* コールバックは、適切なタイミングで呼び出される無名関数で構築されます。
* 以下のような位置引数が必要であることに注意してください
* https://keras.io/ja/callbacks/

## コールバックを作成
* keras.callbacks.Callbackを拡張することで出来る

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=784, kernel_initializer='uniform'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

history = LossHistory()
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, callbacks=[history])

print(history.losses)

## 例：モデルのチェックポイント


In [ ]:
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Dense(10, input_dim=784, kernel_initializer='uniform'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

'''
バリデーションロスが減少した場合に，各エポック終了後，モデルの重みを保存します
'''
checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)
model.fit(x_train,
          y_train,
          batch_size=128,
          epochs=20,
          verbose=0,
          validation_data=(X_test, Y_test), 
          callbacks=[checkpointer])